# Cryptocurrency Data Downloader
## Download Latest Data (2020-2025) for Multiple Cryptocurrencies

In [1]:
# Install required libraries (run once)
# !pip install yfinance pandas numpy requests

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import time
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Current date: {datetime.now().strftime('%Y-%m-%d')}")

Libraries imported successfully!
Current date: 2025-11-13


## 1. Define Cryptocurrencies to Download

In [2]:
# List of cryptocurrencies (Add/Remove as needed)
crypto_symbols = [
    'BTC-USD',   # Bitcoin
    'ETH-USD',   # Ethereum
    'ADA-USD',   # Cardano
    'BNB-USD',   # Binance Coin
    'XRP-USD',   # Ripple
    'SOL-USD',   # Solana
    'DOT-USD',   # Polkadot
    'DOGE-USD',  # Dogecoin
    'MATIC-USD', # Polygon
    'AVAX-USD',  # Avalanche
    'LINK-USD',  # Chainlink
    'UNI-USD',   # Uniswap
    'LTC-USD',   # Litecoin
    'ATOM-USD',  # Cosmos
    'XLM-USD',   # Stellar
    'ALGO-USD',  # Algorand
    'VET-USD',   # VeChain
    'TRX-USD',   # Tron
    'FIL-USD',   # Filecoin
    'ETC-USD',   # Ethereum Classic
    'XMR-USD',   # Monero
    'AAVE-USD',  # Aave
    'THETA-USD', # Theta
    'EOS-USD',   # EOS
    'XTZ-USD',   # Tezos
]

# Create mapping of symbols to clean names
crypto_names = {
    'BTC-USD': 'Bitcoin',
    'ETH-USD': 'Ethereum',
    'ADA-USD': 'Cardano',
    'BNB-USD': 'Binance Coin',
    'XRP-USD': 'Ripple',
    'SOL-USD': 'Solana',
    'DOT-USD': 'Polkadot',
    'DOGE-USD': 'Dogecoin',
    'MATIC-USD': 'Polygon',
    'AVAX-USD': 'Avalanche',
    'LINK-USD': 'Chainlink',
    'UNI-USD': 'Uniswap',
    'LTC-USD': 'Litecoin',
    'ATOM-USD': 'Cosmos',
    'XLM-USD': 'Stellar',
    'ALGO-USD': 'Algorand',
    'VET-USD': 'VeChain',
    'TRX-USD': 'Tron',
    'FIL-USD': 'Filecoin',
    'ETC-USD': 'Ethereum Classic',
    'XMR-USD': 'Monero',
    'AAVE-USD': 'Aave',
    'THETA-USD': 'Theta',
    'EOS-USD': 'EOS',
    'XTZ-USD': 'Tezos',
}

print(f"Total cryptocurrencies to download: {len(crypto_symbols)}")
print("\nList:")
for i, symbol in enumerate(crypto_symbols, 1):
    print(f"{i:2d}. {crypto_names.get(symbol, symbol)}")

Total cryptocurrencies to download: 25

List:
 1. Bitcoin
 2. Ethereum
 3. Cardano
 4. Binance Coin
 5. Ripple
 6. Solana
 7. Polkadot
 8. Dogecoin
 9. Polygon
10. Avalanche
11. Chainlink
12. Uniswap
13. Litecoin
14. Cosmos
15. Stellar
16. Algorand
17. VeChain
18. Tron
19. Filecoin
20. Ethereum Classic
21. Monero
22. Aave
23. Theta
24. EOS
25. Tezos


## 2. Set Date Range

In [3]:
# Define date range
START_DATE = '2020-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')  # Today's date

print(f"Downloading data from {START_DATE} to {END_DATE}")
print(f"Total duration: ~{(datetime.now() - datetime.strptime(START_DATE, '%Y-%m-%d')).days} days")

Total duration: ~2143 days


## 3. Download Data Function

In [4]:
def download_crypto_data(symbol, start_date, end_date):
    """
    Download cryptocurrency data from Yahoo Finance
    
    Parameters:
    - symbol: Crypto symbol (e.g., 'BTC-USD')
    - start_date: Start date (YYYY-MM-DD)
    - end_date: End date (YYYY-MM-DD)
    
    Returns:
    - DataFrame with OHLCV data
    """
    try:
        # Download data
        data = yf.download(symbol, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            print(f"  ❌ No data found for {symbol}")
            return None
        
        # Reset index to make Date a column
        data = data.reset_index()
        
        # Rename columns to match your format
        data = data.rename(columns={
            'Date': 'Date',
            'Open': 'Open',
            'High': 'High',
            'Low': 'Low',
            'Close': 'Close',
            'Volume': 'Volume',
            'Adj Close': 'Adj_Close'
        })
        
        # Add metadata columns
        clean_symbol = symbol.replace('-USD', '')
        data['Symbol'] = clean_symbol
        data['Name'] = crypto_names.get(symbol, clean_symbol)
        
        # Calculate Market Cap (approximation: Close * Volume)
        # Note: This is not real market cap, just a placeholder
        data['Marketcap'] = data['Close'] * data['Volume']
        
        # Add SNo
        data.insert(0, 'SNo', range(1, len(data) + 1))
        
        # Select and reorder columns to match your format
        columns_order = ['SNo', 'Name', 'Symbol', 'Date', 'High', 'Low', 
                        'Open', 'Close', 'Volume', 'Marketcap']
        data = data[columns_order]
        
        return data
        
    except Exception as e:
        print(f"  ❌ Error downloading {symbol}: {str(e)}")
        return None

print("Download function ready!")

Download function ready!


## 4. Download All Cryptocurrencies

In [5]:
# Create directories
os.makedirs('crypto_data_new', exist_ok=True)
os.makedirs('crypto_data_old', exist_ok=True)  # Backup your old data here

print("Starting download...\n")
print("=" * 70)

downloaded_data = {}
successful_downloads = 0
failed_downloads = 0

for i, symbol in enumerate(crypto_symbols, 1):
    print(f"[{i}/{len(crypto_symbols)}] Downloading {crypto_names.get(symbol, symbol)} ({symbol})...")
    
    # Download data
    df = download_crypto_data(symbol, START_DATE, END_DATE)
    
    if df is not None:
        # Save to CSV
        clean_symbol = symbol.replace('-USD', '')
        filename = f'crypto_data_new/{clean_symbol.lower()}_data.csv'
        df.to_csv(filename, index=False)
        
        downloaded_data[symbol] = df
        successful_downloads += 1
        
        print(f"  ✅ Downloaded {len(df)} records")
        print(f"  💾 Saved to: {filename}")
    else:
        failed_downloads += 1
    
    print()
    
    # Small delay to avoid rate limiting
    time.sleep(0.5)

print("=" * 70)
print(f"\n✅ Successfully downloaded: {successful_downloads}/{len(crypto_symbols)}")
print(f"❌ Failed downloads: {failed_downloads}/{len(crypto_symbols)}")

Starting download...

[1/25] Downloading Bitcoin (BTC-USD)...
  ✅ Downloaded 2143 records
  💾 Saved to: crypto_data_new/btc_data.csv

[2/25] Downloading Ethereum (ETH-USD)...
  ✅ Downloaded 2143 records
  💾 Saved to: crypto_data_new/eth_data.csv

[3/25] Downloading Cardano (ADA-USD)...
  ✅ Downloaded 2143 records
  💾 Saved to: crypto_data_new/ada_data.csv

[4/25] Downloading Binance Coin (BNB-USD)...
  ✅ Downloaded 2143 records
  💾 Saved to: crypto_data_new/bnb_data.csv

[5/25] Downloading Ripple (XRP-USD)...
  ✅ Downloaded 2143 records
  💾 Saved to: crypto_data_new/xrp_data.csv

[6/25] Downloading Solana (SOL-USD)...
  ✅ Downloaded 2043 records
  💾 Saved to: crypto_data_new/sol_data.csv

[7/25] Downloading Polkadot (DOT-USD)...
  ✅ Downloaded 1911 records
  💾 Saved to: crypto_data_new/dot_data.csv

[8/25] Downloading Dogecoin (DOGE-USD)...
  ✅ Downloaded 2143 records
  💾 Saved to: crypto_data_new/doge_data.csv

[9/25] Downloading Polygon (MATIC-USD)...
  ✅ Downloaded 1910 records
  💾 

## 5. Combine All Data into Single File

In [6]:
# Combine all downloaded data
if downloaded_data:
    combined_df = pd.concat(downloaded_data.values(), ignore_index=True)
    
    # Sort by Symbol and Date
    combined_df = combined_df.sort_values(['Symbol', 'Date']).reset_index(drop=True)
    
    # Update SNo
    combined_df['SNo'] = range(1, len(combined_df) + 1)
    
    # Save combined file
    combined_df.to_csv('crypto_data_combined.csv', index=False)
    
    print("\n" + "=" * 70)
    print("COMBINED DATASET SUMMARY")
    print("=" * 70)
    print(f"Total records: {len(combined_df):,}")
    print(f"Cryptocurrencies: {combined_df['Symbol'].nunique()}")
    print(f"Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
    print(f"File saved: crypto_data_combined.csv")
    print("=" * 70)
    
    # Show sample
    print("\nSample data:")
    display(combined_df.head(10))
    
    # Summary by coin
    print("\nRecords per cryptocurrency:")
    summary = combined_df.groupby('Symbol').agg({
        'Date': ['count', 'min', 'max'],
        'Close': ['mean', 'min', 'max']
    }).round(2)
    display(summary)
else:
    print("No data was downloaded successfully.")


COMBINED DATASET SUMMARY
Total records: 52,263
Cryptocurrencies: 25
Date range: 2020-01-01 00:00:00 to 2025-11-12 00:00:00
File saved: crypto_data_combined.csv

Sample data:


Price,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,...,High,Low,Open,Close,Volume,High,Low,Open,Close,Volume
Ticker,,,,,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,,...,EOS-USD,EOS-USD,EOS-USD,EOS-USD,EOS-USD,XTZ-USD,XTZ-USD,XTZ-USD,XTZ-USD,XTZ-USD
0,1,Aave,AAVE,2020-10-02,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Aave,AAVE,2020-10-03,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Aave,AAVE,2020-10-04,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Aave,AAVE,2020-10-05,NaN,NaN,NaN,NaN,NaN,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Aave,AAVE,2020-10-06,NaN,NaN,NaN,NaN,NaN,2.472399e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Aave,AAVE,2020-10-07,NaN,NaN,NaN,NaN,NaN,2.737070e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Aave,AAVE,2020-10-08,NaN,NaN,NaN,NaN,NaN,7.259723e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Aave,AAVE,2020-10-09,NaN,NaN,NaN,NaN,NaN,3.818165e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Aave,AAVE,2020-10-10,NaN,NaN,NaN,NaN,NaN,5.280042e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Records per cryptocurrency:


KeyError: "Column(s) ['Close', 'Date'] do not exist"

## 6. Data Quality Check

In [ ]:
if downloaded_data:
    print("DATA QUALITY REPORT")
    print("=" * 70)
    
    # Check for missing values
    print("\nMissing values per column:")
    print(combined_df.isnull().sum())
    
    # Check for duplicates
    duplicates = combined_df.duplicated(subset=['Symbol', 'Date']).sum()
    print(f"\nDuplicate rows (Symbol + Date): {duplicates}")
    
    # Check data types
    print("\nData types:")
    print(combined_df.dtypes)
    
    # Basic statistics
    print("\nBasic statistics:")
    display(combined_df[['High', 'Low', 'Open', 'Close', 'Volume']].describe())
    
    print("\n" + "=" * 70)

## 7. Next Steps

### ✅ Data Downloaded Successfully!

**What you have now:**
- Individual CSV files in `crypto_data_new/` folder
- Combined dataset: `crypto_data_combined.csv`
- Data from 2020 to present day

**Next steps:**
1. Move your old CSV files to `crypto_data_old/` folder (backup)
2. Run `01_data_processing.ipynb` with the new data
3. Run `02_exploratory_data_analysis.ipynb`
4. Run `03_ml_models.ipynb`

**Need to update data later?**
- Just run this notebook again!
- It will download the latest data automatically